In [ ]:
# automatically reload the package when it is modified
%load_ext autoreload
%autoreload 2

In [ ]:
from mtr.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from mtr.datasets.waymo.waymo_dataset_bc import WaymoDatasetBC as WaymoDataset
# from mtr.datasets.waymo.waymo_dataset import WaymoDataset

from tools.mtr_env import BatchMTREnv
from tools.mtr_inference import MTRInference
from tools.mtr_lightning import MTR_Lightning
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal, MixtureSameFamily, Categorical, Independent

cfg = cfg_from_yaml_file('tools/cfgs/waymo/bc+10_percent_data.yaml', cfg)
dataset = WaymoDataset(cfg.DATA_CONFIG, training=False)
env = BatchMTREnv(1, dataset)

model = MTR_Lightning(cfg)
# model = model.load_from_checkpoint('output/bc/epoch=6-step=10682.ckpt')
_ = model.cuda()

data = dataset[128]
data_batch = dataset.collate_batch([data])

In [ ]:
model.training_step(data_batch, 0)

# Test with dataset

In [ ]:
output_mean = model.model.motion_decoder.output_mean
output_std = model.model.motion_decoder.output_std

In [ ]:
data = dataset[128]
data_batch = dataset.collate_batch([data])
output = model.model(data_batch, False)

In [ ]:
center_gt = output['input_dict']['center_gt'][...,None, :3].cuda()
center_gt = (center_gt - output_mean) / output_std
pred_list = output['pred_list']
pred_scores, pred_ctrls = pred_list[-1]

In [ ]:
center_gt

In [ ]:
best_idx = (pred_ctrls[...,:3] - center_gt).norm(dim=-1, p=2).argmin(dim=-1)
print(best_idx)
pred_ctrls_best = pred_ctrls[torch.arange(pred_ctrls.shape[0]), best_idx]
mode_best = model.model.motion_decoder.build_mode_distribution(pred_ctrls_best)
log_p_best = mode_best.log_prob(center_gt.squeeze(1))
print(log_p_best)
cls_loss = torch.functional.F.cross_entropy(input=pred_scores, target= best_idx, reduction='none')
print(cls_loss)

In [ ]:
mode = model.model.motion_decoder.build_mode_distribution(pred_ctrls)
log_p = mode.log_prob(center_gt)
log_p

In [ ]:
mode, mix, gmm, sample = model.sample(data_batch)
# print(sample)

In [ ]:
mode.log_prob(gt_shift)[0]

# Test in simulation

In [ ]:
env.reset()

In [ ]:
data_batch = env.batch_scene_data
action = model.sample(data_batch)[-1]
env.visualize()
print(action)
_ = env.step(action)

In [ ]:
from mtr.models.utils import common_layers
import torch

class test(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.query = torch.nn.Parameter(torch.randn((2,4)), requires_grad=True)
        self.model = common_layers.build_mlps(c_in = 4, mlp_channels=[8,4], ret_before_act=True, activation='elu')
        
    def forward(self):
        return self.model(self.query)

model = test()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model.zero_grad()
b = model()
loss = torch.abs(b-1).mean()
loss.backward()
optimizer.step()
print(loss, model.query)
